In [ ]:
from U_net import UNet
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from tqdm import tqdm as tqdm
import os
import scipy
import numpy as np
sns.set_style('darkgrid')
import torch
import torchvision
from torchvision import transforms
from  OCTDataset import OCTDataset
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from tqdm import trange
from time import sleep
import torch.nn.functional as F
import sys
import cv2
use_gpu = torch.cuda.is_available()
width = 284
height = 284
width_out = 284
height_out = 284
from skimage.transform import resize

In [38]:
input_path = os.path.join('2015_BOE_Chiu')
subject_path = [os.path.join(input_path, 'Subject_0{}.mat'.format(i)) for i in range(1, 10)] + [os.path.join(input_path, 'Subject_10.mat')]
data_indexes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [39]:
def threhold(x):
    if x < 0.3:
        return 0
    else:
        return 1

threhold = np.vectorize(threhold, otypes=[np.float])

D:\Anaconda3\envs\ImageSegmentation\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [40]:
x = []
y = []
for path in tqdm(subject_path):
    mat = scipy.io.loadmat(path)
    images = mat['images']
    label = mat['manualFluid1']
    images = np.transpose(images, (2, 0, 1))
    label = np.transpose(label, (2, 0, 1))
    label = threhold(label)
    images = resize(images, (images.shape[0], width, height))
    label = resize(label, (label.shape[0], width_out, height_out))
    for idx in data_indexes:
        x += [np.expand_dims(images[idx], 0)]
        y += [np.expand_dims(label[idx], 0)]

x = np.array(x)
y = np.array(y)

 10%|█         | 1/10 [00:05<00:46,  5.15s/it]


KeyboardInterrupt: 

In [ ]:
x.shape, y.shape

In [ ]:
x_train = x[:90]
x_val = x[90:]
y_train = y[:90]
y_val = y[90:]

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
plt.imshow(y_train[12,0])

In [ ]:
x_train = torch.from_numpy(x_train).float()
x_val = torch.from_numpy(x_val).float()
y_train = torch.from_numpy(y_train).long()
y_val = torch.from_numpy(y_val).long()

In [ ]:
batch_size = 9
epochs = 700
epoch_lapse = 50
threshold = 0.5
sample_size = None

In [ ]:
learning_rate = 0.005
unet = UNet(n_channels=1, n_classes=2)
if use_gpu:
    unet = unet.cuda()
    x_train = x_train.cuda()
    x_val = x_val.cuda()
    y_train = y_train.cuda()
    y_val = y_val.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(unet.parameters(), lr = 0.005, momentum=0.9)

In [ ]:
x_train.shape

In [ ]:
def get_val_loss(x_val, y_val):
    x_val = torch.from_numpy(x_val).float()
    y_val = torch.from_numpy(y_val).long()
    if use_gpu:
        x_val = x_val.cuda()
        y_val = y_val.cuda()
    m = x_val.shape[0]
    outputs = unet(x_val)
    # outputs.shape =(batch_size, n_classes, img_cols, img_rows)
    outputs = outputs.permute(0, 2, 3, 1)
    # outputs.shape =(batch_size, img_cols, img_rows, n_classes)
    outputs = outputs.resize(m*width_out*height_out, 2)
    labels = y_val.resize(m*width_out*height_out)
    loss = F.cross_entropy(outputs, labels)
    return loss.data

In [ ]:
def train_step(inputs, labels, optimizer, criterion):
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = unet(inputs)
    # outputs.shape =(batch_size, n_classes, img_cols, img_rows)
    outputs = outputs.permute(0, 2, 3, 1)
    # outputs.shape =(batch_size, img_cols, img_rows, n_classes)
    outputs = outputs.resize(batch_size*width_out*height_out, 2)
    labels = labels.resize(batch_size*width_out*height_out)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
epoch_iter = np.ceil(x_train.shape[0] / batch_size).astype(int)
t = trange(epochs, leave=True)
for _ in t:
    total_loss = 0
    for i in range(epoch_iter):
        batch_train_x = x_train[i * batch_size : (i + 1) * batch_size]
        batch_train_y = y_train[i * batch_size : (i + 1) * batch_size]
        batch_loss = train_step(batch_train_x , batch_train_y, optimizer, criterion)
        total_loss += batch_loss
    if (_+1) % epoch_lapse == 0:
        val_loss = get_val_loss(x_val, y_val)
        print(f"Total loss in epoch {_+1} : {total_loss / epoch_iter} and validation loss : {val_loss}")

In [ ]:
torch.cuda.empty_cache()